In [2]:
# imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
from rdflib.namespace import DC, FOAF
import matplotlib.pyplot as plt
import numpy as np
import networkx.algorithms.community as nx_comm
import networkx as nx
from owlready2 import *


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
from owlready2 import get_ontology
onto = get_ontology("http://example.org/medsur.owl")

In [4]:
class Patient(Thing):
    namespace = onto
    

""" Create Classes and properties of patient Characteristcs""" 
# ---------------------------------------------
class AgeGroup(Thing):
    namespace = onto
    
class OlderThan65(AgeGroup):
    namespace = onto

class Between45And65(AgeGroup):
    namespace = onto
    
class Between25And45(AgeGroup):
    namespace = onto

class Between18And25(AgeGroup):
    namespace = onto
    
class hasAgeGroup(ObjectProperty):
    domain = [Patient]
    range = [AgeGroup]                  # is this range correct if we connect patient directly to olderthan65 group is rdf tripple, else delete range
    namespace = onto
 
# ---------------------------------------------   
class Gender(Thing):
    namespace = onto
    
class hasGender(ObjectProperty):
    domain = [Patient]
    range = [Gender]
    namespace = onto
    
class WeightGroup(Thing):
    namespace = onto
    
class Obese(WeightGroup):
    namespace = onto
    
class Overweight(WeightGroup):
    namespace = onto
    
class NormalWeight(WeightGroup):
    namespace = onto

class Underweight(WeightGroup):
    namespace = onto

class hasWeightGroup(ObjectProperty):   # is this range correct if we connect patient directly to weightgroup is rdf tripple, else delete range
    domain = [Patient]
    range = [WeightGroup]
    namespace = onto
 
# ---------------------------------------------   
#class ID(Thing):
    #namespace = onto
    
#class hasID(ObjectProperty):
    #domain = [Patient]
    #range = [ID]
    #namespace = onto


"""Create Classes and Properties of Symptoms"""

# System Organ Class
class Symptoms(Thing):                                     # TODO: this may need to be gone
    namespace = onto
    
class suffersFrom(ObjectProperty):                         # TODO: this need to be gone
    domain = [Patient]
    range = [Symptoms]
    namespace = onto
    
class SOCCode(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [int]
    namespace = onto
    
class SOCName(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto                                        # TODO: check whether equuivalence is infered
    
class SOCAbbr(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto
    equivalent_to = [SOCCode, SOCName]

class hasSOC(ObjectProperty):
    domain = [Patient]
    range = [Symptoms]
    namespace = onto
     
   
# High Level Group Term
class HLGTCode(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [int]
    namespace = onto
    
class HLGTName(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto
    equivalent_to = [HLGTCode]
    
class hasHLGT(ObjectProperty):
    domain = [Patient]
    range = [Symptoms]
    namespace = onto
   
    
# High Level Term
class HLTCode(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [int]
    namespace = onto
    
class HLTName(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto
    
class PSD(DataProperty, FunctionalProperty): # Primary Source Describtion
    domain = [Symptoms]
    range = [str]
    namespace = onto
    equivalent_to = [HLTCode]
    
class hasHLT(ObjectProperty):
    domain = [Patient]
    range = [Symptoms]
    namespace = onto	
   
    
# Preferred Term
class PTCode(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [int]
    namespace = onto
    
class PTName(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto
    equivalent_to = [PTCode]
    
class hasPT(ObjectProperty):
    domain = [Patient]
    range = [Symptoms]
    namespace = onto


# Lowest Level Term  
class LLTCode(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [int]
    namespace = onto

class LLTName(DataProperty, FunctionalProperty):
    domain = [Symptoms]
    range = [str]
    namespace = onto
    equivalent_to = [LLTCode]
    
class hasLLT(ObjectProperty):
    domain = [Patient]
    range = [Symptoms]
    namespace = onto
    

"""Create Classes and Properties of Compound"""
class Compound(Thing):
    namespace = onto
    
class isGiven(ObjectProperty):
    domain = [Patient]
    range = [Compound]
    namespace = onto

class ACTText(DataProperty, FunctionalProperty):
    domain = [Compound]
    range = [str]
    namespace = onto
    
class ACTCode(DataProperty, FunctionalProperty):
    domain = [Compound]
    range = [int]
    namespace = onto
    
class hasACT(ObjectProperty):
    domain = [Patient]
    range = [Compound]
    namespace = onto    
    
# Generic Drug Name
    
class GenericDrugName(DataProperty, FunctionalProperty):
    domain = [Compound]
    range = [str]
    namespace = onto
    equivalent_to = [ACTText, ACTCode]                      

class Dosis(DataProperty, FunctionalProperty):
    domain = [Compound]
    range = [str]
    namespace = onto

class Type(DataProperty, FunctionalProperty):
    domain = [Compound]
    range = [str]
    namespace = onto
   

"""Create Classes and Properties of Outcome"""	
class Outcome(Thing):
    namespace = onto
    
class hasOutcome(ObjectProperty):
    domain = [Patient]
    range = [Outcome]
    namespace = onto
    
class OutcomeCode(DataProperty, FunctionalProperty):
    domain = [Outcome]
    range = [int]
    namespace = onto
    
class OutcomeText(DataProperty, FunctionalProperty):
    domain = [Outcome]
    range = [str]
    namespace = onto
    equivalent_to = [OutcomeCode]

In [5]:
# load medsur data from output_opiod.xlsx
import pandas as pd
data = pd.read_excel('output_opioid.xlsx')
data.head()

,Unnamed: 0,ID,WorldwideUniqueCaseIdentification,Status,Category,BodyWeight,Height,BMI,BMIGroup,date_received,...,HLTGName,SOCCode,SOCName,SOCAbbreviation,IsCurrent,IsDefaultSOC,Primary Source Description,summary,narrative,Dosage
0,0,1,CF-NL-09-249,40,NaN,NaN,NaN,NaN,NaN,2009-12-14,...,Psychiatric disorders NEC,10037175,Psychiatric disorders,Psych,1,1,she had become dependant of pain killers and s...,Automatic summary of the report by the Netherl...,The following information was published in the...,50MG
1,1,2,CF-NL-09-249,40,NaN,NaN,NaN,NaN,NaN,2009-12-14,...,Therapeutic and nontherapeutic effects (excl t...,10018065,General disorders and administration site cond...,Genrl,1,1,paracetamol and codeine no longer sufficiently...,Automatic summary of the report by the Netherl...,The following information was published in the...,50MG
2,2,3,ES-KYOWAKIRIN-2019BKK009955,40,NaN,NaN,NaN,NaN,NaN,2019-06-19,...,Parathyroid gland disorders,10014698,Endocrine disorders,Endo,1,0,Hypercalcemia,NaN,Case reference number 2019BKK009955 is a solic...,100UG/DO
3,3,4,ES-KYOWAKIRIN-2019BKK009955,40,NaN,NaN,NaN,NaN,NaN,2019-06-19,...,Deliria (incl confusion),10037175,Psychiatric disorders,Psych,1,1,Disorientation,NaN,Case reference number 2019BKK009955 is a solic...,100UG/DO
4,4,5,ES-KYOWAKIRIN-2019BKK009955,40,NaN,NaN,NaN,NaN,NaN,2019-06-19,...,"Bone, calcium, magnesium and phosphorus metabo...",10027433,Metabolism and nutrition disorders,Metab,1,1,Hypercalcemia,NaN,Case reference number 2019BKK009955 is a solic...,100UG/DO


In [6]:
onto.save(file = "medsur.rdf", format = "rdfxml") 
g = rdflib.Graph()
g.parse("medsur.rdf", format="xml")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

Graph g has 152 statements.


In [7]:
# add RDF triples to the ontology
EX = rdflib.Namespace("http://example.org/medsur.owl#")


# only select first 50 rows
df = df[0:50]

for index, row in data.itterows():
    patient = URIRef(EX, row['PatientID'])                      # check whether this is correct, do we have to add thing
    onto.add((patient, RDF.type, EX.Patient))                   # check whether this is correct
    
    # TODO: check whether we have to add the class agegroup to the ontology
    onto.add((patient, EX.hasAgeGroup, data["AgeGroup"]))                # maybe with if statement to filter on groups, groups may need to be defined in ontology
    
    # select age group
    if data["Age"] >= 65:
        onto.add((patient, EX.hasAgeGroup, EX.OlderThan65))     
    elif data["Age"] >= 45:
        onto.add((patient, EX.hasAgeGroup, EX.Between45And65))   
    elif data["Age"] >= 25:
        onto.add((patient, EX.hasAgeGroup, EX.Between25And45))    
    elif data["Age"] >= 18:
        onto.add((patient, EX.hasAgeGroup, EX.Between18And25))
        
    # TODO: do we have to do somethingh with the class weightgroup? or can we just use the data property?    
    if data["BMI"] >= 30:
        onto.add((patient, EX.hasWeightGroup, EX.Obese))
    elif data["BMI"] >= 25:
        onto.add((patient, EX.hasWeightGroup, EX.Overweight))
    elif data["BMI"] >= 18.5:
        onto.add((patient, EX.hasWeightGroup, EX.Normal))
    elif data["BMI"] < 18.5:
        onto.add((patient, EX.hasWeightGroup, EX.Underweight))
    
    onto.add((patient, EX.hasGender, data["Gender"]))
    
    onto.add((patient, EX.hasOutcome, data["Outcome"]))
    
    #onto.add((patient, EX.suffersFrom, EX.Symptoms)) # TODO: is this necessary? or can we lose the symptoms class?
    onto.add((patient, EX.hasPT, data["PTName"]))
    onto.add((patient, EX.hasHLT, data["HLTName"]))
    onto.add((patient, EX.hasHLGT, data["HLGTName"]))
    onto.add((patient, EX.hasLLT, data["LLTName"]))
    onto.add((patient, EX.hasACT, data["ACTText"]))
    
    # TODO: how to do the equivalent classes
    
    
    

    
    
    
    
    
    # may be inferred
    
    

    
    


AttributeError: 'DataFrame' object has no attribute 'itterows'